In [1]:
import numpy as np

In [2]:
class Atom:
    def __init__(self,x,y,z,element,charge,notused,fx,fy,fz,hydrogens,oxygen):
        self.x=float(x)
        self.y=float(y)
        self.z=float(z)
        self.element=str(element)
        self.charge=float(charge)
        self.notused=int(float(notused))
        self.fx=float(fx)
        self.fy=float(fy)
        self.fz=float(fz)
        self.hydrogens=hydrogens
        self.oxygen=oxygen
    
    @classmethod
    def from_lammps(cls, element, x, y, z,notused):
        obj = object.__new__(cls)
        obj.x = float(x)
        obj.y = float(y)
        obj.z = float(z)
        obj.element = str(element)
        obj.notused = notused
        obj.hydrogens=[]
        obj.oxygen=None
        return obj
    
    def scan_vicinity(self,config,cutoff,boxl):
        pivot=np.array([self.x,self.y,self.z])
        config=np.array(config)
        for ind,i in enumerate(config):
            #print(str(ind)+" element= "+i.element,type(i.notused))
            if(i.element=="O"):
                i_vector=np.array([i.x,i.y,i.z])
                #print("O at"+str(ind))
                direction=pivot-i_vector
                direction[0]=direction[0]-(boxl*np.round(direction[0]/boxl))
                direction[1]=direction[1]-(boxl*np.round(direction[1]/boxl))
                direction[2]=direction[2]-(boxl*np.round(direction[2]/boxl))
                dist=np.linalg.norm(direction)
                if(dist<cutoff):
                    #can set notused = 1 here. Need to ponder
                    
                    return False
        return True
    
    def calc_dist(self,pivot,i_vector,boxl):
        direction=pivot-i_vector
        direction[0]=direction[0]-(boxl*np.round(direction[0]/boxl))
        direction[1]=direction[1]-(boxl*np.round(direction[1]/boxl))
        direction[2]=direction[2]-(boxl*np.round(direction[2]/boxl))
        dist=np.linalg.norm(direction)
        return dist
    
    def scan_H_OH(self,config,cutoff,boxl):
        pivot=np.array([self.x,self.y,self.z])
        config=np.array(config)
        for ind,i in enumerate(config):
            if(i.element=="O"):
                i_vector=np.array([i.x,i.y,i.z])
                dist=self.calc_dist(pivot,i_vector,boxl)
                if(dist<cutoff):
                    if(len(i.hydrogens)<2):
                        i.hydrogens.append((i,dist))
                        #print("before sorting: ",i.hydrogens)
                        i.hydrogens.sort(key=lambda tup: tup[1])
                        #print("after sorting: ",i.hydrogens)
                        self.oxygen=i
                    elif(dist<i.hydrogen[-1]):
                        i.hydrogen[-1][0].oxygen=None
                        i.hydrogen.pop()
                        i.hydrogen.append((i,dist))
                        #print("before sorting: ",i.hydrogens)
                        i.hydrogens.sort(key=lambda tup: tup[1])
                        #print("after sorting: ",i.hydrogens)
                        self.oxygen=i
                        
                        
        
                

In [3]:
def parse(f_name):
    config=[]
    with open(f_name,'r') as f:
        temp=[]
        for i in f.readlines():
            i=str(i)
            #print(i)
            if("begin" in i):
                temp=[]
            elif("end" in i):
                config.append(temp)
                break # need to remove for a whole file
            elif(i.startswith("atom")):
                a=i.split()
                temp.append(Atom(a[1],a[2],a[3],a[4],a[5],a[6],a[7],a[8],a[9]))
    #for j in config:
    #    for i in j:
    #        print(i.element,type(i.element))
    return config                  

In [4]:
def parse_lammps_output(f_name):
    atoms=['O','H']
    config=[]
    with open(f_name,'r') as f:
        temp=[]
        for ind,i in enumerate(f.readlines()):
            i=str(i)
            if("ITEM: TIMESTEP" in i):
                if(ind>0):
                    config.append(temp)
                temp=[]
            elif(any(i.startswith(a) for a in atoms)):
                a=i.split()
                #print(a)
                temp.append(Atom.from_lammps(a[0],a[1],a[2],a[3],0))
    return config


In [5]:
def pH(proton_count,boxl,Na):
    print("Proton Count = "+str(proton_count))
    n=proton_count/Na
    vol=1000*(boxl**3)
    #Molarity=n/vol
    #print("molarity=",Molarity)
    return -1*(np.log10(n)-np.log10(vol))

In [6]:
from sys import argv as a
import numpy as np
import matplotlib.pyplot as plt

def main():
    cutoff=1.15 # 2.173185 # needs to be atomic units since the coordinates are in atomic units
    boxl_atomic=46.937#23.465110
    Na=6.0221409e+23
    atomic_to_angstron=0.5291772083
    boxl_angstron=boxl_atomic*atomic_to_angstron
    boxl_meters=boxl_angstron**-8
    
    
    #config=parse("../training-set/input.data")
    config=parse_lammps_output("./out6.lammpstrj")
    #print(config[0])
    
    
    config=config[:100]
    print("config size",np.shape(config))
    protons=[]
    
    
    for ind,j in enumerate(config):
        proton_count=0
        oh_count=0
        h2o_count=0
        for ind2,i in enumerate(j):
            if(i.element=="H"):
                #print("H at"+str(ind2))
                i.scan_H_OH(j,cutoff,boxl_atomic)
        for ind2,i in enumerate(j):
            if(i.element=="H"): 
                if(i.oxygen==None):
                    proton_count+=1
            elif(i.element=="O" and len(i.hydrogens)==1):
                oh_count+=1
            elif(i.element=="O" and len(i.hydrogens)==2):
                h2o_count+=1
                
        total=proton_count+2*oh_count+3*h2o_count
        protons.append((proton_count,oh_count,h2o_count,total))
    print(protons)
    ph=[]
    for i,prot in enumerate(protons):
        ph_temp=pH(prot[0],boxl_atomic*(10**-10),Na)
        poh_temp=pH(prot[1],boxl_atomic*(10**-10),Na)
        #print(i,". pH = ",ph_temp)
        ph.append((ph_temp,poh_temp))
    return protons,ph
    
    
    
    #    print("pH of config-"+str(ind)+" = "+str(pH(proton_count,boxl_meters,Na)))

In [7]:
if __name__ == "__main__":
    protons,ph=main()
    protons=np.array(protons)
    ph=np.array(ph)
    print("protons by config= ",protons)
    print("pH by config= ",ph)
 

config size (100, 192)
[(7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (8, 8, 56, 192), (19, 15, 47, 190), (19, 15, 47, 190), (19, 15, 47, 190), (19, 15, 47, 190), (19, 15, 47, 190), (19, 15, 47, 190), (19, 15, 47, 190), (10, 6, 56, 190), (10, 6, 56, 190), (10, 6, 56, 190), (10, 6, 56, 190), (10, 6, 56, 190), (10, 6, 56, 190), (9, 7, 56, 191), (9, 7, 56, 191), (9, 7, 56, 191), (9, 7, 56, 191), (8, 6, 57, 191), (9, 5, 57, 190), (9, 5, 57, 190), (9, 5, 57, 190), (9, 5, 57, 190), (9, 5, 57, 190), (9, 5, 57, 190), (7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (7, 7, 57, 192), (9, 9, 55, 192), (9, 9, 55, 192), (9, 9, 55, 192), (9, 9, 55, 192), (9, 9, 55, 192), (8, 8, 56, 192), (8, 8, 56, 192), (8, 8, 56, 192), (8, 8, 56, 192), (8, 8, 56, 192), (8, 8, 56, 192), (8, 8, 56, 192), (8, 8, 56, 192), (8, 8, 56, 192), (8, 8, 56, 192), (8, 8, 56, 192), (8, 

TypeError: 'NoneType' object is not iterable

In [ ]:
plt.title("number of protons in configurations")
plt.xlabel('Config Index')
plt.ylabel('Number of protons')
plt.plot(np.arange(len(protons)),protons)
plt.show()

In [ ]:
plt.title("Frequency count of protons in a config")
plt.xlabel("proton count")
plt.ylabel("configuration count")
plt.hist(protons, bins=np.arange(protons.min(), protons.max()+1))


In [ ]:
plt.title("pH by configurations")
plt.xlabel('Config Index')
plt.ylabel('pH')
plt.plot(np.arange(len(ph)),ph)
plt.show()

In [ ]:
disassociation const 